In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
import time

In [3]:
import syft as sy  # import the Pysyft library
hook = sy.TorchHook(torch)  # hook PyTorch ie add extra functionalities to support Federated Learning
vw1 = sy.VirtualWorker(hook, id="vw1")  # define remote worker vw1
vw2 = sy.VirtualWorker(hook, id="vw2")  # and vw2
vw3 = sy.VirtualWorker(hook, id="vw3")
vw4 = sy.VirtualWorker(hook, id="vw4")
vw5 = sy.VirtualWorker(hook, id="vw5")
vw6 = sy.VirtualWorker(hook, id="vw6")
vw7 = sy.VirtualWorker(hook, id="vw7")
vw8 = sy.VirtualWorker(hook, id="vw8")
vw9 = sy.VirtualWorker(hook, id="vw9")
vw0 = sy.VirtualWorker(hook, id="vw0")


In [4]:
#vw_crypto = sy.VirtualWorker(hook, id='vw_crypto' )
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

In [5]:
compute_nodes = [vw0, vw1, vw2, vw3, vw4, vw5, vw6, vw7, vw8, vw9]

In [6]:
epochs = 10
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = epochs
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()
use_cuda = not args.no_cuda and torch.cuda.is_available()
torch.manual_seed(args.seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [7]:
federated_train_loader = sy.FederatedDataLoader( # this is now a FederatedDataLoader 
    datasets.FashionMNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((vw1, vw2, vw3, vw4, vw5, vw6, vw7, vw8, vw9, vw0)), # we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# No aggregation here

In [9]:
def train(args, model, device, federated_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # now it is a distributed dataset
        model.send(data.location) # send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.item()))

In [10]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [11]:
%%time
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "fmnist_cnn.pt")

Train Epoch: 1 [0/60032 (0%)]	Loss: 2.312420
Train Epoch: 1 [1920/60032 (3%)]	Loss: 2.123379
Train Epoch: 1 [3840/60032 (6%)]	Loss: 1.819058
Train Epoch: 1 [5760/60032 (10%)]	Loss: 1.408664
Train Epoch: 1 [7680/60032 (13%)]	Loss: 1.095940
Train Epoch: 1 [9600/60032 (16%)]	Loss: 1.141997
Train Epoch: 1 [11520/60032 (19%)]	Loss: 0.956914
Train Epoch: 1 [13440/60032 (22%)]	Loss: 0.939769
Train Epoch: 1 [15360/60032 (26%)]	Loss: 1.041346
Train Epoch: 1 [17280/60032 (29%)]	Loss: 0.871910
Train Epoch: 1 [19200/60032 (32%)]	Loss: 0.881196
Train Epoch: 1 [21120/60032 (35%)]	Loss: 0.893526
Train Epoch: 1 [23040/60032 (38%)]	Loss: 0.961888
Train Epoch: 1 [24960/60032 (42%)]	Loss: 0.829886
Train Epoch: 1 [26880/60032 (45%)]	Loss: 0.923244
Train Epoch: 1 [28800/60032 (48%)]	Loss: 0.791858
Train Epoch: 1 [30720/60032 (51%)]	Loss: 0.730345
Train Epoch: 1 [32640/60032 (54%)]	Loss: 0.639516
Train Epoch: 1 [34560/60032 (58%)]	Loss: 0.746028
Train Epoch: 1 [36480/60032 (61%)]	Loss: 0.722750
Train Epoch:

Train Epoch: 6 [1920/60032 (3%)]	Loss: 0.314785
Train Epoch: 6 [3840/60032 (6%)]	Loss: 0.340663
Train Epoch: 6 [5760/60032 (10%)]	Loss: 0.422981
Train Epoch: 6 [7680/60032 (13%)]	Loss: 0.252347
Train Epoch: 6 [9600/60032 (16%)]	Loss: 0.424351
Train Epoch: 6 [11520/60032 (19%)]	Loss: 0.441650
Train Epoch: 6 [13440/60032 (22%)]	Loss: 0.210421
Train Epoch: 6 [15360/60032 (26%)]	Loss: 0.485312
Train Epoch: 6 [17280/60032 (29%)]	Loss: 0.434715
Train Epoch: 6 [19200/60032 (32%)]	Loss: 0.425537
Train Epoch: 6 [21120/60032 (35%)]	Loss: 0.286713
Train Epoch: 6 [23040/60032 (38%)]	Loss: 0.256307
Train Epoch: 6 [24960/60032 (42%)]	Loss: 0.276407
Train Epoch: 6 [26880/60032 (45%)]	Loss: 0.415449
Train Epoch: 6 [28800/60032 (48%)]	Loss: 0.336917
Train Epoch: 6 [30720/60032 (51%)]	Loss: 0.434616
Train Epoch: 6 [32640/60032 (54%)]	Loss: 0.487135
Train Epoch: 6 [34560/60032 (58%)]	Loss: 0.409950
Train Epoch: 6 [36480/60032 (61%)]	Loss: 0.413578
Train Epoch: 6 [38400/60032 (64%)]	Loss: 0.432112
Train E

In [12]:
vw0

<VirtualWorker id:vw0 #objects:7>

In [13]:
compute_nodes[0]

<VirtualWorker id:vw0 #objects:7>